In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from os import listdir
from os.path import join, isfile, exists
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Configuration
data_path = "../data/training/"  # Change to "../data/validation/" if needed
w, h = 256, 256  # Image dimensions

# Get list of available files
bins_files = [f for f in listdir(data_path) if isfile(join(data_path, f)) and f.endswith('.bins')]


print(f"Found {len(bins_files)} .bins files")

Found 199 .bins files


In [4]:
def load_ultrasonic_scan(filename):
    """Load a complete ultrasonic scan with labels and metadata"""
    bins_path = join(data_path, filename)
    labels_path = join(data_path, filename.replace('.bins', '.labels'))
    jsons_path = join(data_path, filename.replace('.bins', '.jsons'))

    # Load images
    data = np.fromfile(bins_path, dtype=np.uint16).astype('float32')
    data -= data.mean()
    data /= data.std() + 0.0001
    images = np.reshape(data, (-1, w, h, 1), 'C')

    # Load labels
    labels = np.loadtxt(labels_path) if exists(labels_path) else np.zeros((images.shape[0], 2))
    if len(labels.shape) == 1:
        labels = labels.reshape(-1, 2)

    # Load metadata
    metadata = None
    if exists(jsons_path):
        try:
            with open(jsons_path, 'r') as f:
                metadata = json.load(f)
        except:
            pass

    return images, labels, metadata



In [12]:
# Select a file to visualize (change index to see different files)
file_index = 0  # Change this to view different files (0 to len(bins_files)-1)
selected_file = bins_files[file_index]

images, labels, metadata = load_ultrasonic_scan(selected_file)

# Visualize the first image
image_index = 70  # Change to view different images from the file

img_2d = images[image_index][:, :, 0]
label = labels[image_index]

has_crack = label[0] > 0.5
crack_size = label[1]

fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        z=img_2d,
        colorscale='Gray',
        showscale=True,
        colorbar=dict(title="Amplitude"),
        hovertemplate='X: %{x}<br>Y: %{y}<br>Amplitude: %{z:.3f}<extra></extra>'
    )
)

title_text = f"Ultrasonic Scan: {selected_file}<br>"
title_text += f"<b>{'🔴 DEFECT DETECTED' if has_crack else '✅ NO DEFECT'}</b><br>"
title_text += f"Crack Size: {crack_size:.2f} mm"

fig.update_layout(
    title=title_text,
    xaxis_title="Width (pixels)",
    yaxis_title="Height (pixels)",
    width=800,
    height=700,
    template='plotly_white'
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)

fig.show()

print(f"\nImage {image_index + 1} of {len(images)}")
print(f"Status: {'DEFECT' if has_crack else 'CLEAN'}")
print(f"Crack Size: {crack_size:.2f} mm")


Image 71 of 100
Status: CLEAN
Crack Size: 0.00 mm
